# Example Calculation
## Client: INTERNAL
---
## Project: develop hydraulic tools
## Calc: CALC-002, hydraulic sample calculations
---
## Authentication
> Stamp, Permit
---
## Revision History
|Revision | Date | Description | By | Reviewer|
| :-------| :----|:------------|:---|:--------|
|       | | DRAFT | KCD |  |
|         |          |            |     |    |

---

## Introduction

In [1]:
# dependancies
using Pkg
#Pkg.add("DataFrames")
#Pkg.add("GridInterpolations")
#Pkg.add("Test")
Pkg.add(PackageSpec(url="https://github.com/kevindorma/Hydraulics.jl"))

using DataFrames
#using GridInterpolations
#using Test
using CSV
using ExcelFiles
#using Hydraulics.jl
using Hydraulics




  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/kevindorma/Hydraulics.jl`
  Updating git-repo `https://github.com/kevindorma/Hydraulics.jl`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


┌ Info: Precompiling Hydraulics [05f54161-5451-43a6-b8bd-93e4e26dee24]
└ @ Base loading.jl:1273


List a common utility table.

In [2]:
Hydraulics.IDmm

,NPS,Schedule,Idmm
,Float64,String,Float64
1,0.125,OD,10.287
2,0.25,OD,13.716
3,0.375,OD,17.145
4,0.5,OD,21.336
5,0.75,OD,26.67
6,1.0,OD,33.401
7,1.25,OD,42.164
8,1.5,OD,48.26
9,2.0,OD,60.325


## Format of input data
Input data is required the following tables.
* fluidList: this tabulates fluid density and viscosity for a given fluid. Used for sizing and hydraulic calculations.

Sizing
* Put the sizing information in the table lineSizing. A spreadsheet is very convenient becauase it allows one to use descriptors for properties (such as fluid type, pipe material, pipe schedule).

Hydraulics
* Need a table (lines) for describing overall conditions for each line segment.
* also need a table (fittingList) for listing all of the piping components in each line segment.

A spreadsheet is very convenient for tabulating the input data. Dropdown menus are used to ensure inputs are correct (fluid type, NPS, pipe schedule). Then the hydraulic calculations are performed using the input data in the spreadsheet.



In [3]:
# these are the specific data
lineSizing = DataFrame(load("inputHydraulicsExample.xlsx", "sizing"))
lineHydraulics = DataFrame(load("inputHydraulicsExample.xlsx", "hydraulics"))
fluidList = DataFrame(load("inputHydraulicsExample.xlsx", "fluidList"))
fittingList = DataFrame(load("inputHydraulicsExample.xlsx", "fittingList"));

## Check input tables for inconsistancies

In [4]:
Hydraulics.checkLineList(lineHydraulics,fluidList)

,message,entry,Segment,NPS,SCHED,material,fluidName
,String,Int64,String,Float64,String,String,String
1,Schedule not found in row,1,A,10.0,STD,Commercial steel,water
2,Schedule not found in row,2,B,8.0,STD,Commercial steel,propane


In [5]:
Hydraulics.pipeRoughness;

## Line Sizing

The spreadsheet hydraulicDescription_2.xls contains the data needed for line sizing.

In [6]:
myLineSize = Hydraulics.getLineSize(lineSizing, fluidList)

,Segment,mmDP100,mmErosion,mmNeeded,Schedule
,String,Float64,Float64,Float64,String
1,A,293.178,314.414,314.414,STD
2,B,66.0907,60.752,66.0907,STD
3,C,53.3495,21.1511,53.3495,STD
4,D,44.0575,27.306,44.0575,STD


In [7]:
chosenLineSizes = Hydraulics.getListLargerNPS(myLineSize)

,Segment,NPS,Schedule,IDmm
,String,Float64,String,Float64
1,A,14.0,STD,336.54
2,B,3.0,STD,77.9
3,C,2.5,STD,62.6
4,D,2.0,STD,52.5


In [8]:
Hydraulics.incrementLineSize(3,chosenLineSizes,1)
chosenLineSizes

,Segment,NPS,Schedule,IDmm
,String,Float64,String,Float64
1,A,14.0,STD,336.54
2,B,3.0,STD,77.9
3,C,3.0,STD,77.9
4,D,2.0,STD,52.5


In [9]:
Hydraulics.sizing2hydraulics(lineSizing, chosenLineSizes)

,Segment,Description,LineTag,PnID,NPS,Schedule,Material,fluidName
,String,String,String,String,Float64,String,String,Any
1,A,Test line,L-001,PID-007,14.0,STD,Commercial steel,propane
2,B,Testline2,L-002,PID-007,3.0,STD,Commercial steel,water
3,C,Testline3,L-003,PID-007,3.0,STD,Commercial steel,water
4,D,Testline3,L-004,PID-007,2.0,STD,Commercial steel,water


In [10]:
Hydraulics.sizing2fittingList(lineSizing, chosenLineSizes)

,Segment,fittingType,num_length_m,comment,revision
,String,String,Float64,String,String
1,A,PIPE,100.0,from line sizing,A
2,B,PIPE,100.0,from line sizing,A
3,C,PIPE,100.0,from line sizing,A
4,D,PIPE,100.0,from line sizing,A


## Hydraulic calculations

First, take a look at the input data.

In [11]:
fluidList

,fluidName,rho_kgm3,mu_mPas,Comment
,String,Float64,Float64,String?
1,water,1000.0,0.99,missing
2,propane,500.0,0.1,missing
3,Air_1,8.47359,0.015,700 kPaa - 15 C


In [12]:
lineHydraulics

,Segment,Description,LineTag,PnID,NPS,Schedule,Material,fluidName
,String,String,String,String,Float64,String,String,String
1,A,one line,L-001,optional,10.0,STD,Commercial steel,water
2,B,another line,L-002,optional,8.0,STD,Commercial steel,propane


In [13]:
fittingList

,Segment,fittingType,num_length_m,comment,revision
,String,String,Float64,String?,String?
1,A,EL45-THD-STD,2.0,missing,missing
2,A,PIPE,55.0,smurf,B
3,B,PIPE,100.0,missing,missing


Get pressure drop for each segment.

In [14]:
myDP = Hydraulics.getSegmentDP(lineHydraulics,fittingList,fluidList)

,Segment,segmentK,DPkpa,inletP,outletP
,String,Float64,Float64,Float64,Float64
1,A,4.26643,0.915928,100.0,99.0841
2,B,7.83933,2.09113,100.0,97.9089
